# BDD hybride par ChromaDB

# Explications

Les bases de données vectorielles hybrides, comme **ChromaDB**, combinent **des vecteurs** (embeddings) pour la recherche sémantique avec **des données classiques** (métadonnées) pour le filtrage et l'organisation. Contrairement aux bases de données classiques (SQL), elles sont optimisées pour les requêtes **approximate nearest neighbor (ANN)** et les recherches **hybrides**.

---

## Structure des collections

- 🔹 **Collections vs Tables** :
  - En **SQL**, on utilise des **tables** avec des colonnes et des types fixes.
  - En **ChromaDB**, on utilise des **collections**, qui stockent des **vecteurs**, des **IDs**, et des **métadonnées JSON**.
  
- 🔹 **Données stockées** dans chaque collection :
  - **Embeddings** : représentations numériques (vecteurs) des données pour la recherche sémantique.
  - **IDs uniques** : identifiants pour récupérer des éléments.
  - **Métadonnées** : informations complémentaires sous forme de dictionnaire JSON.

- 🔹 **Absence de schéma rigide** :
  - Contrairement aux bases SQL, **les collections sont flexibles** : pas besoin de définir à l'avance les colonnes ou types de données.
  - Les **métadonnées peuvent varier d’un élément à l’autre**.

---

## Lien entre les collections

- 🔗 **Pas de relations directes comme en SQL** (pas de `JOIN`) → On simule les relations avec des **IDs croisés ("foreign keys")**.

- 📌 **Stratégies pour gérer les relations** :
  - Ajouter un **champ ID de référence** dans les métadonnées (ex: `"author_id": "auteur_1"` pour lier une image à un auteur).
  - Effectuer des **requêtes en deux étapes** :
    1. Rechercher un élément dans une collection.
    2. Utiliser son ID pour récupérer les données associées dans une autre collection.

  - Exemple d'association **1-N** (un auteur a plusieurs images) :

    ```python
    # Récupérer toutes les images d'un auteur donné
    collection.get(where={"author_id": "auteur_1"})
    ```

  - 📍 Contrairement à SQL, pas de **contrainte d’intégrité référentielle** → Il faut gérer manuellement la suppression et la mise à jour des liens.

---

## Métadonnées

- 🏷️ **Stockées en JSON**, flexibles, et utilisées pour le **filtrage rapide**.
- 🔍 **Utilisées pour des recherches hybrides** :
  - **Recherche vectorielle** pour la similarité sémantique.
  - **Filtrage classique** sur des attributs spécifiques.
  
- 📊 **Avantages des métadonnées** :
  - Permettent d’ajouter des **catégories, tags, sources, relations** aux données vectorielles.
  - Accélèrent la recherche en limitant l’espace de recherche (ex: **ne chercher que dans une catégorie donnée**).

- ✅ **Exemple de requête hybride** (trouver les 3 images les plus proches d’un vecteur, mais uniquement dans la catégorie "animal") :

  ```python
  collection.query(
      query_embeddings=[[0.1, 0.2, 0.3]],
      n_results=3,
      where={"category": "animal"}
  )


# 2 approches possibles

## structure classique d'une BDD

### Imports

In [9]:
import chromadb

### Création de la base/client

In [10]:
# Création du client et de la base locale
client = chromadb.PersistentClient(path="./chroma_db")


### Création des collections

In [11]:
# Création d'une collection
detection = client.get_or_create_collection("detection")
image = client.get_or_create_collection("image")
video = client.get_or_create_collection("video")

### Suppression des collections

In [12]:
# Suppression d'une collection
#client.delete_collection("collection")

### Ajout de données à une collection

In [13]:
# TODO : ajout de catégories obligatoires pour les collections

# Ajout de documents avec vecteurs + métadonnées
image.add(
    ids=["1"],
    embeddings=[[0.1, 0.2, 0.3]],
    metadatas=[{"num_frame": "1", "nom_image": "frame001.jpg"}]
)

### Affichage de l'ensemble

In [14]:
# Récupérer toutes les données (ids, embeddings, metadatas)
all_data = image.get()

# Afficher tout le contenu
for i in range(len(all_data["ids"])):
    print(f"🔹 ID: {all_data['ids'][i]}")
    print(f"🧠 Embedding: {all_data['embeddings'][i]}")
    print(f"📌 Métadonnées: {all_data['metadatas'][i]}")
    print("-" * 40)


🔹 ID: 1


TypeError: 'NoneType' object is not subscriptable

### Requêtes

In [ ]:
# Définition du vecteur de requête
query_vector = [0.15, 0.25, 0.35]

# Recherche hybride (similitude vectorielle + filtre sur la catégorie)
results = collection.query(
    query_embeddings=[query_vector],
    n_results=1,  # Nombre de résultats souhaités
    where={"category": "tech"}  # Filtrage par métadonnée
)

print("🔍 Résultats de la recherche :", results)


🔍 Résultats de la recherche : {'ids': [['1']], 'embeddings': None, 'documents': [[None]], 'uris': None, 'data': None, 'metadatas': [[{'category': 'tech', 'title': 'Doc1'}]], 'distances': [[0.007499998360872616]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


## regroupement de toutes les variables en 1 table

In [15]:
import chromadb

In [16]:
# Création du client et de la base locale
client = chromadb.PersistentClient(path="./chroma_db")

In [17]:
# Création d'une collection
detection = client.get_or_create_collection("detection")

In [ ]:
# TODO : ajout de catégories obligatoires pour les collections

# Ajout de documents avec vecteurs + métadonnées
detection.add(
    ids=["1"],
    embeddings=[[0.1, 0.2, 0.3]],
    metadatas=[{
        "nom_image": "frame001.jpg",
        "num_frame": "1", 
        "nom_video": "video001.mp4",
        "nb_frames_tot": "150",
        "fps": "15",
        "resolution": "1920x1080",
        "class_name": "person",
        "x":"400",
        "y":"200",
        "height":"400",
        "width":"100"}]
)

# REFLEXION : on a une ligne par objet détecté, alors qu'il faudrait une ligne par frame 


In [ ]:
# Récupérer toutes les données (ids, embeddings, metadatas)
all_data = image.get()

# Afficher tout le contenu
for i in range(len(all_data["ids"])):
    print(f"🔹 ID: {all_data['ids'][i]}")
    print(f"🧠 Embedding: {all_data['embeddings'][i]}")
    print(f"📌 Métadonnées: {all_data['metadatas'][i]}")
    print("-" * 40)


In [ ]:
# Définition du vecteur de requête
query_vector = [0.15, 0.25, 0.35]

# Recherche hybride (similitude vectorielle + filtre sur la catégorie)
results = collection.query(
    query_embeddings=[query_vector],
    n_results=1,  # Nombre de résultats souhaités
    where={"category": "tech"}  # Filtrage par métadonnée
)

print("🔍 Résultats de la recherche :", results)
